In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
iris = pd.read_csv('/Users/nanyueming/Documents/学习/python/机器学习与量化投资/机器学习模型/iris.data' , 
                   header = None , names = ['x1','x2','x3','x4','type'])

In [3]:
iris.head()

,x1,x2,x3,x4,type
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


用决策树对数据集进行拟合可以使用sklearn包

参考：https://blog.csdn.net/healingwounds/article/details/83786729

In [4]:
from sklearn import tree

随机划分训练集和测试集

In [5]:
x = iris[['x1','x2','x3','x4']]
y = iris['type']
num = x.shape[0] # 样本总数
ratio = 7/3 # 划分比例，训练集数目:测试集数目
num_test = int(num/(1+ratio)) # 测试集样本数目
num_train = num -  num_test # 训练集样本数目
index = np.arange(num) # 产生样本标号
np.random.shuffle(index) # 洗牌
x_test = x.iloc[index[:num_test]] # 取出洗牌后前 num_test 作为测试集
y_test = y.iloc[index[:num_test]]
x_train = x.iloc[index[num_test:]] # 剩余作为训练集
y_train = y.iloc[index[num_test:]]

In [6]:
clf = tree.DecisionTreeClassifier()

In [7]:
clf.fit(x_test , y_test)

DecisionTreeClassifier()

In [8]:
clf.predict(x_test)

array(['Iris-versicolor', 'Iris-virginica', 'Iris-setosa', 'Iris-setosa',
       'Iris-virginica', 'Iris-versicolor', 'Iris-setosa', 'Iris-setosa',
       'Iris-versicolor', 'Iris-setosa', 'Iris-setosa', 'Iris-versicolor',
       'Iris-versicolor', 'Iris-setosa', 'Iris-versicolor',
       'Iris-virginica', 'Iris-virginica', 'Iris-virginica',
       'Iris-versicolor', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa',
       'Iris-setosa', 'Iris-setosa', 'Iris-virginica', 'Iris-virginica',
       'Iris-versicolor', 'Iris-virginica', 'Iris-virginica',
       'Iris-versicolor', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa',
       'Iris-virginica', 'Iris-virginica', 'Iris-virginica',
       'Iris-virginica', 'Iris-versicolor', 'Iris-versicolor',
       'Iris-virginica', 'Iris-virginica', 'Iris-versicolor',
       'Iris-versicolor', 'Iris-versicolor', 'Iris-setosa'], dtype=object)

计算熵

In [9]:
def CalEntropy(data):
    n = len(data)
    labels = list(set(data))
    
    entropy = 0.0
    for label in labels:
        prob = float( data[data==label].count() / n )
        entropy -= prob * np.log(prob)
    
    return entropy

计算信息增益

In [10]:
def CalInfogain(df , feature):
    ls = list(set(df[feature]))
    
    NewEntropy = 0.0
    for i in range(len(ls)):
        data = df[df[feature] == ls[i]].type
        NewEntropy += float(len(data) / len(df)) * CalEntropy(data)
    
    infogain = CalEntropy(df.type) - NewEntropy  
    
    return infogain

信息增益比

In [11]:
def CalInfogainRatio(df , feature):
    
    infogain = CalInfogain(df , feature)
    entropy = CalEntropy(df[feature])
    
    gr = infogain / entropy
    
    return gr

获取最大信息增益的特征

In [12]:
def GetBestFeature(df):
    BestInfoGain = 0.0
    for i in range(len(df.columns) - 1):
        feature = df.columns[i]
        infogain = CalInfogain(df , feature)
        
        if (infogain > BestInfoGain):
            BestInfoGain = infogain
            BesFeature = feature
    return BesFeature

In [13]:
GetBestFeature(iris)

'x3'

# ID3算法

ID3算法利用信息增益判断节点

In [14]:
for i in list(set(iris.x3)):
    df = iris[iris.x3 == i]
    if len(set(df.type)) > 1:
        f = GetBestFeature(df)
        print(f)

x1
x2
x1
x1
x1
